In [1]:
import pickle
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

In [2]:
import re
import numpy as np


# вспомогательные функции для генерации промпта

def scenarios_to_str(scenarios):
    s = ''
    for i, dct in enumerate(scenarios):
        s += f"{i}) {dct['description']}\nSteps: {dct['steps']}\n\n"
    return s

def context_to_str(context):
    s = ''
    for i, part in enumerate(context):
        s += f"\n=== Part {i} ===\n{part[0]}\n"
    return s


# генерация промпта

def generate_prompt(code, scenarios, context):
    return f"""
Your task is to select EXACTLY the context parts needed to implement the test scenarios for the provided code.

**Rules:**
1. Analyze the code under test and test scenarios carefully
2. Select context parts that contain:
   - Data structures, required for implementing test scenarios
   - Functions / methods of classes, required for implementing test scenarios
   - Any other information, required for implementing test scenarios
3. Exclude:
   - Unrelated components (even if similar)
   - Low-level implementation details
   - Duplicate information
4. Return ONLY Python list of selected part numbers, like [1, 3, 4]

**Code to test:**
{code}

**Test scenarios ({len(scenarios)} total):**
{scenarios_to_str(scenarios)}

**Context parts (0-{len(context)-1}):**
{context_to_str(context)}

Remember: Your response MUST be ONLY the list, e.g. [1, 3, 5]
"""

def generate_prompt_by_data(d):
    return generate_prompt(d['code'], d['scenarios'], d['context'])

prompts = [generate_prompt_by_data(d) for d in data]
print(len(prompts))
print(prompts[0])

16

Your task is to select EXACTLY the context parts needed to implement the test scenarios for the provided code.

**Rules:**
1. Analyze the code under test and test scenarios carefully
2. Select context parts that contain:
   - Data structures, required for implementing test scenarios
   - Functions / methods of classes, required for implementing test scenarios
   - Any other information, required for implementing test scenarios
3. Exclude:
   - Unrelated components (even if similar)
   - Low-level implementation details
   - Duplicate information
4. Return ONLY Python list of selected part numbers, like [1, 3, 4]

**Code to test:**
package org.springframework.samples.petclinic.customers.web;
@RequestMapping("/owners")
@RestController
@Timed("petclinic.owner")
@Slf4j
class OwnerResource {
    private final OwnerRepository ownerRepository;
    private final OwnerEntityMapper ownerEntityMapper;
        @PostMapping
    @ResponseStatus(HttpStatus.CREATED)
    public Owner createOwner(@V

In [3]:

# сколько токенов позволено максимум сгенерировать модели
def calculate_max_new_tokens(tokenizer, N: int, reserve_tokens=100) -> int:
    """Вычисляет максимальную длину генерации для списка всех возможных индексов"""
    # Создаем максимально возможный список [0, 1, ..., N-1]
    max_list_str = str(list(range(N)))
    # Токенизируем эту строку и берем длину
    return len(tokenizer.encode(max_list_str)) + reserve_tokens  # Добавляем запас


# парсинг вывода модели
def parse_model_output(output_text: str) -> list[int]:
    """Парсит вывод модели и возвращает список чисел, обрабатывая ошибки"""
    # Ищем первый вхождение паттерна списка Python
    match = re.search(r'\[([\d,\s]+)\]', output_text)
    if not match:
        return None
    
    try:
        # Извлекаем числа из найденного списка
        numbers = [int(num) for num in match.group(1).split(',')]
        return np.array(numbers)
    except (ValueError, AttributeError):
        return None


# запуск модели
def run_model(model, tokenizer, prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = len(inputs['input_ids'][0])

    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    new_tokens = outputs[0][input_length:]  # новосгенерированные токены
    output_text = tokenizer.decode(new_tokens, skip_special_tokens=True)    # сгенерированный моделью ответ

    return parse_model_output(output_text), output_text

In [6]:
# выбираем не занятую видеокарту
device = 'cuda:5'

# код с квантизацией
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# загрузка модели
# model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
# model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)


model_names = [
    # "deepseek-ai/deepseek-coder-6.7b-instruct",
    "codellama/CodeLlama-7b-Instruct-hf",
    # "huggyllama/llama-7b",
    # "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct",
    
]

preds = {}
for model_name in model_names:
    print(f'loading "{model_name}"...')
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, quantization_config=quantization_config, device_map=device)

    preds[model_name] = []
    print(f'start inference "{model_name}"')
    for i, (prompt, d) in enumerate(zip(prompts, data)):
        max_new_tokens = calculate_max_new_tokens(tokenizer, len(d['context']), reserve_tokens=10)
        try:
            output_idx, output_text = run_model(model, tokenizer, prompt, max_new_tokens)
        except:
            output_idx, output_text = None, None
        print(i, output_idx, output_text)
        preds[model_name].append((output_idx, output_text))

    del model

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

loading "codellama/CodeLlama-7b-Instruct-hf"...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


start inference "codellama/CodeLlama-7b-Instruct-hf"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

11 None 
12 None 


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


13 None 


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


14 None 
15 None 


In [7]:
model_name = model_names[0]
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, quantization_config=quantization_config, device_map=device)

_, text_output = run_model(model, tokenizer, "Tell me about yourself", 1000)
print(text_output)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


.

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user123456789 I'm not sure what you mean by "me".

Comment: @user1234567

In [5]:
with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)